In [6]:
# 🚀 AI Tutor - Clean Working DataSci GPU Tutor (RAG + LLM + Game Mode)
import os
import pickle
import faiss
import torch
import gradio as gr
import numpy as np
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import requests
import random
import difflib
import re
from types import SimpleNamespace
import ast
import base64
from io import BytesIO

# Try importing plotly, fallback gracefully if not available
try:
    import plotly.graph_objects as go
    PLOTLY_AVAILABLE = True
except ImportError:
    print("⚠️ Plotly not available. Install with: pip install plotly")
    PLOTLY_AVAILABLE = False
    go = None
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode


# --- CONFIG ---
# EMBED_MODEL = "/scratch/ntiwar12/huggingfacesentence-transformers/all-MiniLM-L6-v2"
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
#"BAAI/bge-m3"
#"sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "NousResearch/Hermes-2-Pro-Mistral-7B"
#"deepseek-ai/deepseek-coder-6.7b-instruct"
#"microsoft/phi-4"
# LLM_MODEL = "microsoft/DialoGPT-small"  # Much smaller model
#EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# LLM_MODEL = "MiniMaxAI/SynLogic-7B"
# LLM_MODEL = "HuggingFaceH4/zephyr-7b-beta"
 
#"meta-llama/Meta-Llama-3-8B"
#"allenai/digital-socrates-13b"
#"/scratch/ntiwar12/huggingface/hub/models--WizardLM--WizardCoder-Python-34B-V1.0/snapshots/897fc6d9e12136c68c441b2350d015902c144b20/"
#"allenai/digital-socrates-13b"


#"NousResearch/Hermes-2-Pro-Mistral-7B"
#https://huggingface.co/tecosys/Nutaan-RL1
#https://huggingface.co/knowledgator/Qwen-encoder-0.5B
#https://huggingface.co/knowledgator/Llama-encoder-1.0B
CHUNK_FILE = "chunks.pkl"
INDEX_FILE = "faiss.index" 
URLS_FILE = "custom_urls.txt"

# Enhanced documentation system
DOC_LINKS = {
    # Core Libraries
    'pandas': 'https://pandas.pydata.org/docs/',
    'scikit-learn': 'https://scikit-learn.org/stable/',
    'rapids': 'https://rapids.ai/',
    'cudf': 'https://docs.rapids.ai/api/cudf/stable/',
    'cupy': 'https://docs.cupy.dev/en/stable/',
    'pytorch': 'https://pytorch.org/docs/stable/',
    'tensorflow': 'https://www.tensorflow.org/',
    'cuml': 'https://docs.rapids.ai/api/cuml/stable/',
    
    # Data Science
    'dsml_pdf': 'https://people.smp.uq.edu.au/DirkKroese/DSML/DSML.pdf',
    'awesome_datascience': 'https://github.com/academic/awesome-datascience',
    'gfg_ds_beginners': 'https://www.geeksforgeeks.org/data-science/data-science-for-beginners/',
    
    # GPU Acceleration
    'cudf_pandas': 'https://rapids.ai/cudf-pandas/',
    'polars_gpu': 'https://rapids.ai/polars-gpu-engine/',
    'cuml_accel': 'https://rapids.ai/cuml-accel/',
    'nx_cugraph': 'https://rapids.ai/nx-cugraph/',
    'hagedorn2022': 'https://proceedings.mlr.press/v185/hagedorn22a/hagedorn22a.pdf',
    'nvidia_tesla_whitepaper': 'https://www.nvidia.com/docs/io/116711/sc11-nv-tesla.pdf',
    'cuda_blog': 'https://developer.nvidia.com/blog/tag/cuda/',
    'libcxxgpu_pdf': 'https://www.seas.upenn.edu/~delozier/docs/libcxxgpu.pdf',
    'gpu_accel_slides': 'https://dlsyscourse.org/slides/12-gpu-acceleration.pdf',
    'gpucad_paper': 'https://yibolin.com/publications/papers/GPUCAD_ICCAD2020_Lin.pdf',
    'gpucad_slides': 'https://yibolin.com/publications/papers/GPUCAD_ICCAD2020_Lin.slides.pdf',
    'spie_gpu_sample': 'https://www.spiedigitallibrary.org/samples/SL34.pdf',
    'gpu_to_web': 'https://www.khronos.org/assets/uploads/developers/library/2012-the-graphical-web/GPU-to-the-web_Sep2012.pdf',
    
    # RAG Retrieval
    'langgraph_agentic_rag': 'https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/',
    
    # Basics
    'gpu_compare': 'https://www.nvidia.com/en-us/geforce/graphics-cards/compare/',
    'gpu_hierarchy': 'https://www.tomshardware.com/reviews/gpu-hierarchy,4388.html'
}



embedder = SentenceTransformer(EMBED_MODEL, device='cuda')

print(f"🤖 Loading {LLM_MODEL}")
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    trust_remote_code=True, device_map="auto", torch_dtype="auto")


llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    return_full_text=False
) 

def stream_llm_response(prompt, max_new_tokens=1000, temperature=0.7):
    """Generate streaming response from LLM"""
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            # Initialize generation parameters
            input_ids = inputs["input_ids"]
            attention_mask = inputs["attention_mask"]
            
            # Generate tokens one by one
            for _ in range(max_new_tokens):
                outputs = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=1,
                    do_sample=True,
                    temperature=temperature,
                    pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id else tokenizer.pad_token_id,
                    return_dict_in_generate=True,
                    output_scores=True
                )
                
                # Get the new token
                new_token_id = outputs.sequences[0][-1].item()
                
                # Decode the new token
                new_token = tokenizer.decode([new_token_id], skip_special_tokens=True)
                
                # Check for end of sequence
                if new_token_id == tokenizer.eos_token_id:
                    break
                
                # Update inputs for next generation
                input_ids = outputs.sequences
                attention_mask = torch.cat([
                    attention_mask, 
                    torch.ones((1, 1), device=model.device)
                ], dim=1)
                
                yield new_token
                
    except Exception as e:
        print(f"❌ Streaming generation failed: {e}")
        yield "Sorry, I encountered an error while generating the response."

print("✅ LLM loaded and ready!")

# === RAG UTILITIES ===

def read_urls_from_txt(path):
    """Read URLs from a text file"""
    try:
        with open(path, "r") as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"⚠️ URLs file not found: {path}. Creating default URLs...")
        create_default_urls_file(path)
        return read_urls_from_txt(path)

def create_default_urls_file(path):
    """Create a default URLs file with common data science resources"""
    default_urls = [
        "https://pandas.pydata.org/docs/",
        "https://docs.rapids.ai/api/cudf/stable/",
        "https://rapids.ai/",
        "https://scikit-learn.org/stable/",
        "https://pytorch.org/docs/stable/",
        "https://www.tensorflow.org/",
        "https://developer.nvidia.com/blog/tag/cuda/",
        "https://people.smp.uq.edu.au/DirkKroese/DSML/DSML.pdf"
    ]
    with open(path, "w") as f:
        for url in default_urls:
            f.write(url + "\n")
    print(f"✅ Created default URLs file: {path}")

def fetch_text_from_url(url):
    """Fetch and clean text content from a URL"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (RAG Tutor Bot)'}
        resp = requests.get(url, headers=headers, timeout=15)
        
        # Filter out non-text content types
        content_type = resp.headers.get("Content-Type", "")
        if not content_type.startswith("text/") and "html" not in content_type:
            print(f"⚠️ Skipping non-text URL: {url} (type={content_type})")
            return ""
        
        # Parse HTML content safely
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tag in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            tag.decompose()
        return soup.get_text(separator="\n").strip()
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return ""

def chunk_text(text, chunk_size=500, overlap=50):
    """Split text into overlapping chunks"""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

def load_all_chunks(urls):
    """Load and chunk text from all URLs"""
    all_chunks = []
    for url in urls:
        print(f"📥 Processing: {url}")
        txt = fetch_text_from_url(url)
        if txt:
            chunks = chunk_text(txt)
            all_chunks.extend(chunks)
            print(f"✅ Added {len(chunks)} chunks from {url}")
        else:
            print(f"⚠️ No content from {url}")
    return all_chunks

def embed_and_index(chunks):
    """Create embeddings and FAISS index"""
    if not chunks:
        print("⚠️ No chunks to embed")
        return None
    
    print(f"🔄 Creating embeddings for {len(chunks)} chunks...")
    vecs = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(vecs))
    print(f"✅ Created FAISS index with {len(chunks)} chunks")
    return index

def query_rag(query, index, chunks, k=3):
    """Query the RAG system"""
    if index is None or not chunks:
        return "", ""
    
    q_vec = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(np.array(q_vec), k)
    retrieved_chunks = [chunks[i] for i in I[0] if i < len(chunks)]
    return "\n".join(retrieved_chunks), "Custom Knowledge Base"

def load_cache():
    """Load cached chunks and index"""
    if os.path.exists(CHUNK_FILE) and os.path.exists(INDEX_FILE):
        try:
            with open(CHUNK_FILE, "rb") as f:
                chunks = pickle.load(f)
            index = faiss.read_index(INDEX_FILE)
            print(f"✅ Loaded cached RAG with {len(chunks)} chunks")
            return chunks, index
        except Exception as e:
            print(f"❌ Error loading cache: {e}")
            return None, None
    return None, None

def save_cache(chunks, index):
    """Save chunks and index to cache"""
    try:
        with open(CHUNK_FILE, "wb") as f:
            pickle.dump(chunks, f)
        faiss.write_index(index, INDEX_FILE)
        print(f"✅ Cached RAG with {len(chunks)} chunks")
    except Exception as e:
        print(f"❌ Error saving cache: {e}")

def build_or_load_rag():
    """Build or load RAG system"""
    print("🔄 Initializing RAG system...")
    chunks, index = load_cache()
    
    if not chunks or index is None:
        print("🔄 Building new RAG index...")
        urls = read_urls_from_txt(URLS_FILE)
        chunks = load_all_chunks(urls)
        
        if chunks:
            index = embed_and_index(chunks)
            save_cache(chunks, index)
        else:
            print("⚠️ No chunks loaded, RAG will be disabled")
            return [], None
    
    return chunks, index

def is_garbage(text):
    """Check if text is low quality or corrupted"""
    if not text or len(text) < 80:
        return True
    non_ascii = sum(1 for c in text if ord(c) > 126 or ord(c) < 9)
    return non_ascii / len(text) > 0.2

def suggest_related(query):
    """Suggest related questions based on query"""
    related_examples = {
        "gpu": ["What is CUDA?", "cuDF vs pandas?", "cuML for ML tasks?"],
        "pandas": ["How to groupby in pandas?", "pandas vs cuDF?", "Time series in pandas"],
        "scikit-learn": ["What is train_test_split?", "GridSearchCV usage", "PCA in sklearn"],
        "rapids": ["What is RAPIDS?", "cuDF advantages?", "RAPIDS ecosystem?"],
        "machine learning": ["What is supervised learning?", "Cross-validation?", "Feature engineering?"]
    }
    
    suggestions = []
    for keyword, rel_list in related_examples.items():
        if re.search(rf"\b{keyword}\b", query, re.I):
            suggestions = rel_list
            break
    
    if suggestions:
        bullets = "".join([f"- {q}<br>" for q in suggestions])
        return f"<br><br><b>🔎 You might also ask:</b><br>{bullets}"
    return ""

def generate_example_plot():
    """Generate example CPU vs GPU performance plot"""
    if not PLOTLY_AVAILABLE:
        return "<p><i>📊 Chart visualization requires plotly. Install with: <code>pip install plotly</code></i></p>"
    
    try:
        x = [100, 500, 1000, 5000, 10000]
        cpu_times = [0.5, 1.8, 3.2, 15.0, 30.0]
        gpu_times = [0.2, 0.5, 0.8, 2.5, 5.0]

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=cpu_times, mode='lines+markers', name='CPU'))
        fig.add_trace(go.Scatter(x=x, y=gpu_times, mode='lines+markers', name='GPU'))
        fig.update_layout(
            title='CPU vs GPU Execution Time',
            xaxis_title='Input Size',
            yaxis_title='Time (s)',
            legend_title='Processor'
        )

        buffer = BytesIO()
        fig.write_image(buffer, format='png')
        img_base64 = base64.b64encode(buffer.getvalue()).decode()
        return f'<img src="data:image/png;base64,{img_base64}" style="width:100%;max-width:600px;">'
    except Exception as e:
        print(f"❌ Plot generation failed: {e}")
        return "<p><i>📊 Chart generation temporarily unavailable</i></p>"

#from transformers import pipeline

# Example setup (adjust as per your model and tokenizer setup)
#llm_pipeline = pipeline("text-generation", model="meta-llama/Llama-2", device=0)

def polish_response(text):
    """Polish RAG response using LLM with streaming"""
    prompt = f"""
You're an expert Socratic AI tutor for Data Science and GPU Acceleration. Take the content below and improve it into a well-structured explanation that is:

- Friendly and clear
- Easy for students to understand
- Broken into logical sections. Ask questions before you answer. Make them think.
- Uses analogies/examples when helpful
- Suggests any charts or visualizations if relevant
- Includes a short summary or takeaway at the end

Original content:
-------------------------------
{text}
-------------------------------

Now improve and rewrite it in tutor style:
"""
    try:
        # Use streaming for polish_response
        response_parts = []
        for token in stream_llm_response(prompt, max_new_tokens=1500, temperature=0.7):
            response_parts.append(token)
        
        result = "".join(response_parts)
        # Extract the actual response after the prompt
        split = result.split("Now improve and rewrite it in tutor style:")
        return split[-1].strip() if len(split) > 1 else result.strip()
    except Exception as e:
        print(f"❌ polish_response failed: {e}")
        return "Sorry, I couldn't process this content right now."
        
def generate_flashcards_from_rag(query, k=3):
    """Generate flashcards using RAG context"""
    if rag_index is not None and rag_chunks:
        context, source = query_rag(query, rag_index, rag_chunks)
        
        if context and len(context.strip()) > 100:
            prompt = f"""
You're an expert AI tutor for Data Science and GPU Acceleration. From the text below, extract {k} flashcards. Each flashcard should have:
- A question on the front (about a key concept)
- A short, accurate answer on the back

Text:
--------------------
{context}
--------------------

Now generate {k} flashcards in this JSON format:
[
  {{"front": "What is ...?", "back": "It is ..."}},
  ...
]
"""
            try:
                raw = llm_pipeline(prompt, max_new_tokens=200, temperature=0.7)[0]['generated_text']
                match = re.search(r"\[(\s*\{.*?\}\s*,?\s*)+\]", raw, re.DOTALL)
                if match:
                    return ast.literal_eval(match.group(0))
            except Exception as e:
                print(f"❌ RAG Flashcard generation failed: {e}")
    
    # Fallback to original method
    return generate_flashcards(query, k)

# Initialize RAG system
print("🚀 Initializing RAG system...")
rag_chunks, rag_index = build_or_load_rag()

# === CORE FUNCTIONS ===

def generate_contextual_quote(context="learning", user_query=""):
    """Generate contextual inspirational quotes"""
    
    # Try LLM generation first
    if llm_pipeline:
        try:
            topic = extract_topic_from_query(user_query) if user_query else context
            prompt = f"Generate an inspiring quote about {topic} and learning:"
            result = llm_pipeline(prompt, max_new_tokens=50, temperature=0.8)
            if result and len(result) > 0:
                quote = result[0]['generated_text'].strip()
                if len(quote) > 10:
                    return f"🧠 '{quote}' - AI Generated"
        except Exception as e:
            print(f"Quote generation error: {e}")
    
    # Enhanced fallback quotes
    quotes = [
        "💡 'Data is the new oil, but insights are the refined fuel.' - Anonymous",
        "🚀 'The best way to get started is to quit talking and begin doing.' - Walt Disney",
        "🧠 'Machine learning is the last invention that humanity will ever need to make.' - Nick Bostrom",
        "⚡ 'GPU acceleration: Think parallel, compute faster!' - AI Generated",
        "📊 'In data we trust, but insights we must discover!' - AI Generated",
        "🔬 'Every algorithm learns from data, just like we learn from experience!' - AI Generated",
        "🎯 'The goal is to turn data into information, and information into insight.' - Carly Fiorina",
        "🌟 'Artificial intelligence is the new electricity.' - Andrew Ng",
        "⚙️ 'The key to artificial intelligence has always been the representation.' - Jeff Hawkins"
    ]
    return random.choice(quotes)

def extract_topic_from_query(query):
    """Extract main topic from user query"""
    if not query:
        return "learning"
    
    query_lower = query.lower()
    topic_keywords = {
        'gpu computing': ['gpu', 'cuda', 'parallel', 'graphics'],
        'data science': ['data', 'science', 'analytics', 'statistics'],
        'machine learning': ['machine learning', 'ml', 'ai', 'artificial intelligence'],
        'pandas': ['pandas', 'dataframe'],
        'rapids': ['rapids', 'cudf', 'cuml'],
        'programming': ['python', 'code', 'programming']
    }
    
    for topic, keywords in topic_keywords.items():
        if any(keyword in query_lower for keyword in keywords):
            return topic
    return 'learning'

def smart_tutor_answer_streaming(query):
    """Enhanced RAG-powered Socratic tutor responses with streaming"""
    if not query or query.strip() == "":
        yield "🤔 Please ask me a question about data science, machine learning, or GPU computing!"
        return
    
    # Try RAG first
    if rag_index is not None and rag_chunks:
        context, source = query_rag(query, rag_index, rag_chunks)
        
        if context and not is_garbage(context):
            # Stream the polished response
            polished_prompt = f"""
You're a semi-Socratic AI tutor for Data Science and GPU Acceleration. Take the content below and improve it into a well-structured explanation that is:

- Friendly and clear
- Easy for students to understand
- Broken into small logical sections. 
- Ask questions before you answer. Make them think.
- Uses analogies/examples when helpful
- Suggests any charts or visualizations if relevant
- Includes a short summary or takeaway at the end

Original content:
-------------------------------
{context.strip()}
-------------------------------

Now improve and rewrite it in tutor style:
"""
            for token in stream_llm_response(polished_prompt, max_new_tokens=1500, temperature=0.7):
                yield token
            return
    
    # Fallback to LLM streaming for non-RAG responses
    query_lower = query.lower()

    # Generic response with LLM streaming
    if llm_pipeline:
        try:
            prompt = f"""You are a thoughtful, semi-socratic, and concise AI tutor specialized in data science and GPU acceleration.
    
    Your goal is to guide the learner with an engaging and structured explanation. 
    - Start by briefly asking a clarifying or reflective question (Socratic style)
    - Then answer clearly, avoiding jargon. Keep answers short.
    - Use examples or analogies if helpful
    
    Q: {query}
    A:"""
            for token in stream_llm_response(prompt, max_new_tokens=1000, temperature=0.7):
                yield token
            return
        except Exception as e:
            print(f"❌ LLM fallback failed: {e}")

    yield f"""🤔 Let's explore this together:

• What do you already know about {query}?
• How might this relate to data science or computing?
• What would you want to accomplish by understanding this better?

💡 **Hint:** Break down the concept into smaller parts and think about how each works."""

def generate_flashcards(topic, n=3):
    """Generate educational flashcards with RAG enhancement"""
    
    # Try RAG-based generation first
    if rag_index is not None and rag_chunks:
        try:
            context, source = query_rag(topic, rag_index, rag_chunks)
            if context and len(context.strip()) > 100:
                prompt = f"""
Create {n} educational flashcards about {topic} from this content. Format as JSON array:
[{{"front": "question", "back": "answer"}}]

Content:
{context[:1000]}
"""
                result = llm_pipeline(prompt, max_new_tokens=1000, temperature=0.7)
                if result:
                    text = result[0]['generated_text']
                    # Try to extract JSON array
                    match = re.search(r'\[.*\]', text, re.DOTALL)
                    if match:
                        try:
                            cards = ast.literal_eval(match.group(0))
                            if isinstance(cards, list) and len(cards) > 0:
                                return cards[:n]
                        except:
                            pass
        except Exception as e:
            print(f"❌ RAG flashcard generation failed: {e}")
    
    # Fallback flashcards
    flashcard_pools = {
        "cudf": [
            {"front": "What is cuDF?", "back": "cuDF is a GPU DataFrame library with pandas-like API."},
            {"front": "How to convert pandas to cuDF?", "back": "Use cudf.from_pandas(df)"},
            {"front": "Main advantage of cuDF?", "back": "GPU acceleration for large dataset processing"}
        ],
        "pandas": [
            {"front": "What is a DataFrame?", "back": "A 2D labeled data structure with columns of different types."},
            {"front": "How to read CSV in pandas?", "back": "pd.read_csv('filename.csv')"},
            {"front": "How to select a column?", "back": "df['column_name'] or df.column_name"}
        ],
        "rapids": [
            {"front": "What is RAPIDS?", "back": "A suite of GPU-accelerated data science libraries."},
            {"front": "Main RAPIDS libraries?", "back": "cuDF, cuML, cuGraph for DataFrames, ML, and graphs."},
            {"front": "RAPIDS vs traditional tools?", "back": "Same APIs but with GPU acceleration for massive speedups."}
        ],
        "cuda": [
            {"front": "What does CUDA stand for?", "back": "Compute Unified Device Architecture"},
            {"front": "What is CUDA used for?", "back": "Parallel computing on NVIDIA GPUs"},
            {"front": "CUDA vs CPU computing?", "back": "CUDA enables massive parallelization vs sequential CPU processing"}
        ]
    }
    
    # Get cards for topic, with fallback
    cards = flashcard_pools.get(topic.lower(), [
        {"front": f"What is {topic}?", "back": f"{topic} is an important concept in data science."},
        {"front": f"Why learn {topic}?", "back": f"Understanding {topic} improves your data science skills."}
    ])
    
    return cards[:n]

def generate_quiz(topic, n=2):
    """Generate quiz questions"""
    
    # Quiz pools by topic
    quiz_pools = {
        "pandas": [
            {"question": "What is the main data structure in pandas?", "options": ["DataFrame", "Array", "List", "Dict"], "answer_idx": 0},
            {"question": "How do you read a CSV file?", "options": ["pd.read_csv()", "pd.load()", "pd.import()", "pd.open()"], "answer_idx": 0}
        ],
        "cudf": [
            {"question": "cuDF accelerates which library?", "options": ["NumPy", "Pandas", "SciPy", "Matplotlib"], "answer_idx": 1},
            {"question": "cuDF runs on which hardware?", "options": ["CPU", "GPU", "TPU", "FPGA"], "answer_idx": 1}
        ],
        "rapids": [
            {"question": "RAPIDS is developed by?", "options": ["Google", "Facebook", "NVIDIA", "Microsoft"], "answer_idx": 2},
            {"question": "Main RAPIDS component for DataFrames?", "options": ["cuML", "cuDF", "cuGraph", "cuPy"], "answer_idx": 1}
        ]
    }
    
    questions = quiz_pools.get(topic.lower(), [
        {"question": f"What is {topic}?", "options": ["A tool", "A library", "A concept", "All of above"], "answer_idx": 3},
        {"question": f"Why is {topic} important?", "options": ["Performance", "Efficiency", "Scale", "All of above"], "answer_idx": 3}
    ])
    
    return random.sample(questions, min(n, len(questions)))

def generate_coding_puzzle(difficulty="Beginner"):
    """Generate coding puzzles for pandas to cuDF conversion"""
    
    puzzles = {
        "Beginner": {
            "description": "Convert basic DataFrame creation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'a': [1, 2, 3]})\nprint(df)",
            "cpu_time": 1.2,
            "gpu_time": 0.2
        },
        "Intermediate": {
            "description": "Convert groupby operation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'group': ['A', 'B', 'A'], 'value': [1, 2, 3]})\nresult = df.groupby('group').sum()",
            "cpu_time": 3.2,
            "gpu_time": 0.5
        },
        "Advanced": {
            "description": "Convert complex aggregation from pandas to cuDF",
            "cpu_code": "import pandas as pd\ndf = pd.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "gpu_code": "import cudf\ndf = cudf.DataFrame({'cat': ['A', 'B'], 'val1': [1, 2], 'val2': [3, 4]})\nresult = df.groupby('cat').agg({'val1': 'sum', 'val2': 'mean'})",
            "cpu_time": 4.2,
            "gpu_time": 0.6
        }
    }
    
    return puzzles.get(difficulty, puzzles["Beginner"])

# === GRADIO INTERFACE ===
def create_ai_tutor():
    """Create the AI Tutor Gradio interface"""
    
    with gr.Blocks(theme=gr.themes.Soft(), title="🧠 AI Tutor") as app:
        
        # Header
        gr.Markdown("""
        <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:15px; margin-bottom:20px; color:#000;'>
          <h1 style='color:#000; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>🧠 AI Tutor - Complete Learning System</h1>
          <h3 style='color:#000; font-weight:700; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'>📚 Learning Mode • 🎓 Socratic Tutoring • 🎮 Gamified Practice</h3>
          <p style='color:#000; font-weight:600; text-shadow: 1px 1px 2px rgba(0,0,0,0.1);'><b>📚 Structured Learning Paths • 🤔 Guided Discovery • 🎯 Interactive Practice</b></p>
        </div>
        """)
        
        # Add custom CSS for better styling
        app.css = """
        .learning-topic-btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
            color: white !important;
            border: none !important;
            border-radius: 12px !important;
            padding: 15px !important;
            margin: 8px !important;
            font-weight: 600 !important;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4) !important;
            transition: all 0.3s ease !important;
            min-height: 80px !important;
            font-size: 14px !important;
        }
        .learning-topic-btn:hover {
            transform: translateY(-2px) !important;
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.6) !important;
        }
        
        /* Make tab headers more prominent */
        .gradio-tab-nav {
            background: linear-gradient(90deg, #f8fafc, #e2e8f0) !important;
            border-radius: 8px !important;
            padding: 8px !important;
            margin-bottom: 20px !important;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1) !important;
        }
        
        .gradio-tab-nav .tab-nav {
            background: linear-gradient(135deg, #4f46e5, #7c3aed) !important;
            color: white !important;
            font-weight: 700 !important;
            font-size: 16px !important;
            border-radius: 6px !important;
            margin: 4px !important;
            padding: 12px 20px !important;
            box-shadow: 0 2px 4px rgba(79, 70, 229, 0.3) !important;
            border: none !important;
        }
        
        .gradio-tab-nav .tab-nav:hover {
            background: linear-gradient(135deg, #6366f1, #8b5cf6) !important;
            transform: translateY(-1px) !important;
            box-shadow: 0 4px 8px rgba(79, 70, 229, 0.4) !important;
        }
        
        .gradio-tab-nav .tab-nav.selected {
            background: linear-gradient(135deg, #059669, #0d9488) !important;
            box-shadow: 0 4px 12px rgba(5, 150, 105, 0.4) !important;
        }
        """
        
        with gr.Tabs():
            
            # === LEARNING MODE ===
            with gr.Tab("📚 Learning Mode"):
                gr.Markdown("### 🎯 **Choose a Learning Path - From Beginner to Advanced**")
                gr.Markdown("*Select any topic below to get a structured learning journey with resources and related topics*")
                
                # Learning topics with structured paths
                LEARNING_TOPICS = {
                    "Python for Data Science": {
                        "description": "Master Python fundamentals and data science libraries",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "4-6 weeks",
                        "prerequisites": "Basic programming knowledge",
                        "content": """
**📖 Learning Path:**
1. **Python Basics** - Variables, data types, control structures
2. **NumPy** - Numerical computing and arrays
3. **Pandas** - Data manipulation and analysis
4. **Matplotlib/Seaborn** - Data visualization
5. **Jupyter Notebooks** - Interactive development

**🔗 Key Resources:**
• [Python.org Tutorial](https://docs.python.org/3/tutorial/)
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [NumPy User Guide](https://numpy.org/doc/stable/user/)

**🚀 Next Steps:** Machine Learning Fundamentals, Data Visualization
""",
                        "related": ["Machine Learning Fundamentals", "Data Visualization", "Pandas Deep Dive"]
                    },
                    "Pandas Deep Dive": {
                        "description": "Master DataFrame operations, data cleaning, and advanced pandas techniques",
                        "level": "🟡 Intermediate",
                        "duration": "3-4 weeks", 
                        "prerequisites": "Python basics, basic pandas knowledge",
                        "content": """
**📖 Learning Path:**
1. **DataFrame Mastery** - Creation, indexing, selection
2. **Data Cleaning** - Missing values, duplicates, data types
3. **GroupBy Operations** - Aggregation and transformation
4. **Merging & Joining** - Combining datasets
5. **Performance Optimization** - Efficient pandas operations

**🔗 Key Resources:**
• [Pandas Documentation](https://pandas.pydata.org/docs/)
• [Pandas Cookbook](https://pandas.pydata.org/docs/user_guide/cookbook.html)
• [10 Minutes to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html)

**🚀 Next Steps:** GPU Acceleration with cuDF, Time Series Analysis
""",
                        "related": ["GPU Acceleration with cuDF", "Data Visualization", "Time Series Analysis"]
                    },
                    "GPU Acceleration with cuDF": {
                        "description": "Learn GPU-accelerated data processing with RAPIDS cuDF",
                        "level": "🔴 Advanced",
                        "duration": "2-3 weeks",
                        "prerequisites": "Strong pandas knowledge, basic GPU concepts",
                        "content": """
**📖 Learning Path:**
1. **GPU Computing Basics** - Understanding parallel processing
2. **cuDF Introduction** - GPU DataFrames and basic operations
3. **Migration from Pandas** - Converting existing code
4. **Performance Optimization** - Memory management and best practices
5. **Advanced Operations** - Complex aggregations and joins

**🔗 Key Resources:**
• [RAPIDS cuDF Documentation](https://docs.rapids.ai/api/cudf/stable/)
• [cuDF User Guide](https://docs.rapids.ai/api/cudf/stable/user_guide/)
• [RAPIDS Getting Started](https://rapids.ai/start.html)

**🚀 Next Steps:** RAPIDS Ecosystem, Machine Learning with cuML
""",
                        "related": ["RAPIDS Ecosystem", "CUDA Programming", "High-Performance Computing"]
                    },
                    "Machine Learning Fundamentals": {
                        "description": "Core ML concepts, algorithms, and scikit-learn implementation",
                        "level": "🟡 Intermediate",
                        "duration": "6-8 weeks",
                        "prerequisites": "Python, pandas, basic statistics",
                        "content": """
**📖 Learning Path:**
1. **ML Concepts** - Supervised, unsupervised, reinforcement learning
2. **Data Preprocessing** - Feature scaling, encoding, train-test splits
3. **Regression Algorithms** - Linear, polynomial, regularization
4. **Classification** - Logistic regression, decision trees, SVM
5. **Model Evaluation** - Cross-validation, metrics, hyperparameter tuning

**🔗 Key Resources:**
• [Scikit-learn Documentation](https://scikit-learn.org/stable/)
• [Machine Learning Course by Andrew Ng](https://www.coursera.org/learn/machine-learning)
• [Hands-On Machine Learning](https://github.com/ageron/handson-ml2)

**🚀 Next Steps:** Deep Learning, GPU-Accelerated ML with cuML
""",
                        "related": ["Deep Learning Fundamentals", "GPU-Accelerated ML", "Data Science Projects"]
                    },
                    "RAPIDS Ecosystem": {
                        "description": "Complete RAPIDS suite: cuDF, cuML, cuGraph for end-to-end GPU data science",
                        "level": "🔴 Advanced",
                        "duration": "4-5 weeks",
                        "prerequisites": "GPU computing basics, pandas, scikit-learn",
                        "content": """
**📖 Learning Path:**
1. **RAPIDS Overview** - cuDF, cuML, cuGraph, cuPy integration
2. **cuDF Mastery** - Advanced DataFrame operations on GPU
3. **cuML for ML** - GPU-accelerated machine learning algorithms
4. **cuGraph** - Graph analytics and network analysis
5. **End-to-End Workflows** - Complete GPU data science pipelines

**🔗 Key Resources:**
• [RAPIDS.ai Main Site](https://rapids.ai/)
• [RAPIDS Documentation](https://docs.rapids.ai/)
• [RAPIDS Community](https://github.com/rapidsai)

**🚀 Next Steps:** Production Deployment, Multi-GPU Computing
""",
                        "related": ["Multi-GPU Computing", "Production ML Systems", "CUDA Programming"]
                    },
                    "Data Visualization": {
                        "description": "Create compelling data visualizations with matplotlib, seaborn, and plotly",
                        "level": "🟢 Beginner to Intermediate",
                        "duration": "3-4 weeks",
                        "prerequisites": "Python basics, pandas fundamentals",
                        "content": """
**📖 Learning Path:**
1. **Matplotlib Basics** - Plots, figures, axes, customization
2. **Seaborn for Statistics** - Statistical plots and themes
3. **Interactive Plotly** - Dynamic and web-ready visualizations
4. **Advanced Techniques** - Subplots, animations, custom plots
5. **Dashboard Creation** - Streamlit, Dash for interactive apps

**🔗 Key Resources:**
• [Matplotlib Documentation](https://matplotlib.org/stable/contents.html)
• [Seaborn Tutorial](https://seaborn.pydata.org/tutorial.html)
• [Plotly Python Guide](https://plotly.com/python/)

**🚀 Next Steps:** Interactive Dashboards, Business Intelligence
""",
                        "related": ["Interactive Dashboards", "Business Intelligence", "Web Development for Data Science"]
                    }
                }
                
                # Topic selection buttons
                with gr.Row():
                    topic_buttons = []
                    for i, (topic_name, topic_info) in enumerate(LEARNING_TOPICS.items()):
                        if i % 2 == 0 and i > 0:
                            # Start new row every 2 buttons
                            pass
                        
                        color_map = {
                            "🟢": "#dcfce7",  # Green for beginner
                            "🟡": "#fef3c7",  # Yellow for intermediate  
                            "🔴": "#fee2e2"   # Red for advanced
                        }
                        level_color = color_map.get(topic_info["level"][0], "#f3f4f6")
                        
                        btn = gr.Button(
                            f"{topic_name}\n{topic_info['level']}",
                            elem_classes="learning-topic-btn",
                            size="lg"
                        )
                        topic_buttons.append((btn, topic_name))
                
                # Learning content display
                learning_content = gr.Markdown()
                related_topics = gr.Markdown()
                
                def show_learning_content(topic_name):
                    if topic_name not in LEARNING_TOPICS:
                        return "Topic not found!", ""
                    
                    topic = LEARNING_TOPICS[topic_name]
                    
                    content = f"""
# 📚 {topic_name}

**📋 Description:** {topic['description']}

**📊 Level:** {topic['level']} | **⏱️ Duration:** {topic['duration']} | **📚 Prerequisites:** {topic['prerequisites']}

---

{topic['content']}

---
                    """
                    
                    related = f"""
### 🔗 **Related Learning Paths:**
{' • '.join([f"**{rel}**" for rel in topic['related']])}

💡 *Click any topic above to explore these related learning paths!*
                    """
                    
                    return content, related
                
                # Connect buttons to content display
                for btn, topic_name in topic_buttons:
                    btn.click(
                        lambda tn=topic_name: show_learning_content(tn),
                        outputs=[learning_content, related_topics]
                    )
                
                # Search functionality
                with gr.Row():
                    search_topic = gr.Textbox(
                        label="🔍 Search for specific topics",
                        placeholder="e.g., neural networks, time series, NLP, computer vision..."
                    )
                    search_btn = gr.Button("Search Learning Resources")
                
                search_results = gr.Markdown()
                
                def search_learning_resources(query):
                    if not query:
                        return "Please enter a search term!"
                    
                    # Simulate search results with relevant resources
                    query_lower = query.lower()
                    
                    results = []
                    
                    if any(term in query_lower for term in ['neural', 'deep', 'cnn', 'rnn', 'transformer']):
                        results.append("""
**🧠 Deep Learning Resources:**
• [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning)
• [PyTorch Tutorials](https://pytorch.org/tutorials/)
• [TensorFlow Guide](https://www.tensorflow.org/guide)
""")
                    
                    if any(term in query_lower for term in ['nlp', 'text', 'language', 'sentiment']):
                        results.append("""
**📝 Natural Language Processing:**
• [NLTK Documentation](https://www.nltk.org/)
• [spaCy Course](https://course.spacy.io/)
• [Hugging Face Transformers](https://huggingface.co/transformers/)
""")
                    
                    if any(term in query_lower for term in ['time series', 'forecasting', 'temporal']):
                        results.append("""
**📈 Time Series Analysis:**
• [Time Series Analysis Guide](https://www.statsmodels.org/stable/tsa.html)
• [Prophet Forecasting](https://facebook.github.io/prophet/)
• [Time Series with Python](https://github.com/marcopeix/TimeSeriesForecastingInPython)
""")
                    
                    if any(term in query_lower for term in ['computer vision', 'image', 'cv', 'opencv']):
                        results.append("""
**👁️ Computer Vision:**
• [OpenCV Tutorials](https://docs.opencv.org/master/d9/df8/tutorial_root.html)
• [Computer Vision Course](https://www.coursera.org/learn/convolutional-neural-networks)
• [Fast.ai Practical Deep Learning](https://course.fast.ai/)
""")
                    
                    if not results:
                        results.append(f"""
**🔍 Search Results for "{query}":**

*No specific resources found. Try these general resources:*
• [Kaggle Learn](https://www.kaggle.com/learn) - Free micro-courses
• [Coursera Data Science](https://www.coursera.org/browse/data-science)
• [edX MIT Data Science](https://www.edx.org/course/introduction-to-computational-thinking-and-data-science)

*Or refine your search with terms like: machine learning, deep learning, NLP, computer vision, time series*
""")
                    
                    return "\n".join(results)
                
                search_btn.click(search_learning_resources, inputs=search_topic, outputs=search_results)

            # === TUTOR MODE ===
            with gr.Tab("🎓 Tutor Mode"):
                gr.Markdown("### 🤔 **Socratic Learning - Ask Questions, Get Guided Answers**")
                gr.Markdown("*I won't give you direct answers, but I'll guide you to discover the knowledge yourself!*")
                
                with gr.Row():
                    with gr.Column(scale=2):
                        tutor_query = gr.Textbox(
                            label="Ask about Data Science, GPU Computing, or type 'docs' for documentation",
                            placeholder="e.g., What is RAPIDS? How does cuDF work? Why use GPU for data science?",
                            lines=2
                        )
                        tutor_btn = gr.Button("🚀 Get Socratic Guidance", variant="primary")
                    
                    with gr.Column(scale=1):
                        quote_display = gr.Markdown("💡 *Ready to discover knowledge!*")
                
                tutor_answer = gr.Markdown(label="🤔 Socratic Guidance")
                tutor_source = gr.Markdown(label="📚 Source")
                
                # Quick question suggestions
                with gr.Row():
                    quick_questions = [
                        "Why use GPUs for data science?",
                        "How does parallel processing work?", 
                        "What makes cuDF faster than pandas?",
                        "When should I use machine learning?"
                    ]
                    for i, question in enumerate(quick_questions):
                        if i % 2 == 0:
                            with gr.Column():
                                pass
                        quick_btn = gr.Button(f"💭 {question}", size="sm")
                        quick_btn.click(lambda q=question: q, outputs=tutor_query)
                
                def handle_tutor_query_streaming(query):
                    if not query:
                        yield "Please ask a question!", "AI Tutor", "💡 *Ready to help!*"
                        return
                    
                    # Handle docs command
                    if query.lower().strip() in ['docs', 'documentation', 'help']:
                        docs_info = """📚 **Available Documentation:**
• **Core Libraries:** pandas, cuDF, RAPIDS, scikit-learn
• **GPU Computing:** CUDA, NVIDIA resources  
• **Data Science:** Comprehensive guides and tutorials
*Ask any topic-specific question to get relevant documentation!*"""
                        yield docs_info, "Documentation System", "📖 *Knowledge at your fingertips!*"
                        return
                    
                    # Generate contextual quote
                    quote = generate_contextual_quote(extract_topic_from_query(query), query)
                    
                    # Stream the response
                    accumulated_response = ""
                    for token in smart_tutor_answer_streaming(query):
                        accumulated_response += token
                        yield accumulated_response, f"**Source:** AI Tutor (Streaming)", f"🌟 {quote}"
                
                # Update the click handlers to use streaming
                tutor_btn.click(
                    handle_tutor_query_streaming, 
                    inputs=tutor_query, 
                    outputs=[tutor_answer, tutor_source, quote_display],
                    show_progress=True
                )
                tutor_query.submit(
                    handle_tutor_query_streaming, 
                    inputs=tutor_query, 
                    outputs=[tutor_answer, tutor_source, quote_display],
                    show_progress=True
                )
            
            # === GAME MODE ===
            with gr.Tab("🎮 Game Mode"):
                user_state = SimpleNamespace(points=0)
                
                def update_progress():
                    level = user_state.points // 20 + 1
                    bar_width = min((user_state.points % 20) * 5, 100)
                    return f"""<h3>🏆 Level {level} | 🔥 {user_state.points} XP</h3>
                    <div style='background: #e5e7eb; height: 20px; border-radius: 10px;'>
                        <div style='background: linear-gradient(90deg, #4ade80, #22c55e); height: 20px; width: {bar_width}%; border-radius: 10px;'></div>
                    </div>"""
                
                progress = gr.HTML(update_progress())
                
                #with gr.Tabs():
                    
                # === FLASHCARDS ===
                with gr.Tabs():
                    with gr.Tab("🃏 Flashcards"):
                        with gr.Tab("🔧 Generate Flashcards"):
                            gen_query = gr.Textbox(label="Enter a topic (e.g., cuDF, CUDA, RAPIDS)", placeholder="cuDF vs pandas?")
                            gen_btn = gr.Button("✨ Generate Flashcards")
                
                            card_front = gr.HTML()
                            card_back = gr.HTML(visible=False)
                            instruction_msg = gr.Markdown("👆 Choose a card to flip!", visible=False)
                            feedback_msg = gr.Markdown(visible=False)
                
                            with gr.Row():
                                flip_btn1 = gr.Button("✅ Flip Card 1", visible=False)
                                flip_btn2 = gr.Button("🔄 Flip Card 2", visible=False)
                
                            reset_btn = gr.Button("🔁 Reset", visible=False)
                
                            flash_idx = gr.State(0)
                            show_answer = gr.State(False)
                            flashcards_state = gr.State([])
                
                            import ast
                            import random
                            import re
                            def generate_flashcards_from_rag(query, index, chunks, k=3):
                                context, source = query_rag(query, index, chunks)
                            
                                if not context or len(context.strip()) < 100:
                                    return [{"front": "Could not find content", "back": "Try another topic.", "wrong": "This is a placeholder wrong answer."}]
                            
                                prompt = f"""
                            You're an expert AI tutor. From the following technical text, generate exactly {k} multiple-choice style flashcards in **valid JSON**.
                            
                            Each flashcard must be a dictionary with:
                            - "front": the question
                            - "back": the correct answer
                            - "wrong": a wrong but plausible distractor
                            
                            Example format:
                            
                            [
                              {{
                                "front": "Why is cuDF faster than pandas?",
                                "back": "It uses GPU acceleration.",
                                "wrong": "Because it uses more RAM."
                              }},
                              ...
                            ]
                            
                            DO NOT add explanation, markdown, or anything else. Just output a valid JSON list of 3 objects.
                            
                            Text:
                            ------------------------
                            {context}
                            ------------------------
                            """
                            
                                try:
                                    raw = llm_pipeline(prompt)[0]['generated_text']
                            
                                    # DEBUG: Log raw output when things fail
                                    print("\n🔍 RAW OUTPUT FROM MODEL:\n", raw[:500], "...\n")
                            
                                    json_text = re.search(r'\[\s*{.*?}\s*\]', raw, re.DOTALL)
                                    if json_text:
                                        return ast.literal_eval(json_text.group())
                            
                                except Exception as e:
                                    print(f"❌ Flashcard generation failed: {e}")
                            
                                # fallback
                                return [{"front": "Could not parse flashcards", "back": "", "wrong": ""}]
                            def style_card(text):
                                return f"""
                <div style='font-size:18px;padding:10px;border:2px solid #ccc;border-radius:10px;background:#fefefe;transition:transform 0.5s;transform-style: preserve-3d;'>
                {text}
                </div>"""
                
                            def flashcard_from_query(topic, idx):
                                if not topic or topic.strip() == "":
                                    return (
                                            style_card("**Q:** Please enter a topic to generate flashcards."),
                                            "",
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=False),
                                            gr.update(visible=True, value="<span style='color:red; font-size:18px;'>❗ Please enter a topic to begin.</span>"),
                                            idx, False, [], True, False ) # empty flashcard state
                                cards = generate_flashcards_from_rag(topic, rag_index, rag_chunks)
                                #cards = generate_flashcards_from_rag(topic)
                                idx = (idx + 1) % len(cards)
                                card = cards[idx]
                                buttons = [("✅ Flip Card 🧐 ", True), ("✅ Flip Card 🤓", False)]
                                random.shuffle(buttons)
                                return (
                                    style_card(f"**Q:** {card['front']}"),
                                    "",
                                    gr.update(visible=True, value=buttons[0][0]),
                                    gr.update(visible=True, value=buttons[1][0]),
                                    gr.update(visible=True),
                                    gr.update(visible=False, value=""),
                                    gr.update(visible=True, value="<span style='color:green; font-size:18px;'>👆 Choose a card to flip!</span>"),
                                    idx, False, cards, buttons[0][1], buttons[1][1]
                                )
                
                            def flip_card(idx, cards, correct):
                                card = cards[idx % len(cards)]
                                front_html = style_card(f"**Q:** {card['front']}")
    
                                if correct:
                                    back_html = style_card(f"✅ Answer: {card['back']}")
                                else:
                                    back_html = gr.update(visible=False)  # hide back card for wrong pick
                                wrong_msgs = [
                                    "🚫 Not quite! As they say, 'Data is the new oil'—but you've hit a dry well. Try the other card!",
                                    "💡 'In God we trust, all others bring data.' Sadly, this card didn't bring it. Flip the other one!",
                                    "🧠 Oops! ‘The greatest value of a picture is when it forces us to notice what we never expected to see.’ You missed it—check the other card.",
                                    "⚙️ 'GPU acceleration turns hours into seconds' — but this pick cost you a moment. Try the other one!",
                                    "📊 'Torture the data long enough, and it will confess to anything.' This card stayed silent. Flip the other!",
                                    "❌ Wrong Pick! 'Without data, you're just another person with an opinion.' Try the other card for some real answers!",
                                    "🚀 'GPUs don’t guess—they compute at scale.' Your guess here missed. Go try the other card!",
                                    "🔄 Oops! You’ve hit a cold cache. Try the other card for a GPU-hot answer.",
                                    "💭 'Data science is the art of turning data into insight'—but this card had none. Flip the other one!"
                                ]
                                msg = "✅ Nailed it! Great job." if correct else random.choice(wrong_msgs)
                            
                                return (
                                    front_html,
                                    back_html,
                                    gr.update(visible=True),
                                    gr.update(visible=True, value=f"<span style='font-size:18px; color:#b00020;'>{msg}</span>"),      # ✅ THIS is the fix
                                    gr.update(visible=False),                # hide instruction
                                    idx,
                                    True,
                                    cards
                                )            
                            btn1_correct = gr.State(True)
                            btn2_correct = gr.State(False)
                
                            gen_btn.click(
                                flashcard_from_query,
                                inputs=[gen_query, flash_idx],
                                outputs=[card_front, card_back, flip_btn1, flip_btn2, reset_btn, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state, btn1_correct, btn2_correct]
                            )
                
                            flip_btn1.click(
                                flip_card,
                                inputs=[flash_idx, flashcards_state, btn1_correct],
                                outputs=[card_front, card_back, card_back, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state]
                            )
                
                            flip_btn2.click(
                                flip_card,
                                inputs=[flash_idx, flashcards_state, btn2_correct],
                                outputs=[card_front, card_back, card_back, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state]
                            )
                
                            reset_btn.click(
                                flashcard_from_query,
                                inputs=[gen_query, flash_idx],
                                outputs=[card_front, card_back, flip_btn1, flip_btn2, reset_btn, feedback_msg, instruction_msg, flash_idx, show_answer, flashcards_state, btn1_correct, btn2_correct]
                            )
                        
                    # === QUIZ ===
                    with gr.Tab("❓ Quiz"):
                        quiz_topic = gr.Textbox(label="Quiz Topic", placeholder="e.g., pandas, cuDF, RAPIDS")
                        gen_quiz_btn = gr.Button("🧠 Generate Quiz")
                        
                        quiz_q1 = gr.Markdown()
                        quiz_r1 = gr.Radio(choices=[], label="Answer:", visible=False)
                        quiz_q2 = gr.Markdown()
                        quiz_r2 = gr.Radio(choices=[], label="Answer:", visible=False)
                        
                        submit_btn = gr.Button("Submit Quiz", visible=False)
                        quiz_result = gr.Markdown()
                        quiz_state = gr.State([])
                        
                        def generate_quiz_questions(topic):
                            if not topic:
                                return "Enter a topic!", gr.update(visible=False), "", gr.update(visible=False), gr.update(visible=False), []
                            
                            questions = generate_quiz(topic, 2)
                            return (
                                f"**Q1:** {questions[0]['question']}",
                                gr.update(choices=questions[0]['options'], visible=True, value=None),
                                f"**Q2:** {questions[1]['question']}",
                                gr.update(choices=questions[1]['options'], visible=True, value=None),
                                gr.update(visible=True),
                                questions
                            )
                        
                        def evaluate_quiz(ans1, ans2, questions):
                            if not questions:
                                return "Quiz not loaded!"
                            
                            correct = 0
                            result_parts = []
                            
                            # Check Q1
                            correct_ans1 = questions[0]['options'][questions[0]['answer_idx']]
                            if ans1 and ans1 == correct_ans1:
                                correct += 1
                                result_parts.append(f"✅ **Q1:** Correct! ({ans1})")
                            else:
                                result_parts.append(f"❌ **Q1:** Wrong. You answered: {ans1 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans1}")
                            
                            # Check Q2  
                            correct_ans2 = questions[1]['options'][questions[1]['answer_idx']]
                            if ans2 and ans2 == correct_ans2:
                                correct += 1
                                result_parts.append(f"✅ **Q2:** Correct! ({ans2})")
                            else:
                                result_parts.append(f"❌ **Q2:** Wrong. You answered: {ans2 or 'None'}")
                                result_parts.append(f"    💡 **Correct answer:** {correct_ans2}")
                            
                            user_state.points += correct * 5
                            
                            # Overall result
                            score_emoji = "🎉" if correct == 2 else "👍" if correct == 1 else "📚"
                            result_parts.insert(0, f"{score_emoji} **Final Score: {correct}/2 correct! +{correct*5} XP**")
                            result_parts.append(f"\n🏆 **Total XP:** {user_state.points}")
                            
                            return "\n".join(result_parts)
                        
                        gen_quiz_btn.click(generate_quiz_questions, inputs=quiz_topic,
                                         outputs=[quiz_q1, quiz_r1, quiz_q2, quiz_r2, submit_btn, quiz_state])
                        submit_btn.click(evaluate_quiz, inputs=[quiz_r1, quiz_r2, quiz_state], outputs=quiz_result)
                    
                    # === CODING PUZZLES ===
                    with gr.Tab("💻 Coding"):
                        difficulty = gr.Radio(choices=["Beginner", "Intermediate", "Advanced"], 
                                            value="Beginner", label="Difficulty")
                        gen_puzzle_btn = gr.Button("🎯 Generate Puzzle")
                        
                        puzzle_desc = gr.Markdown()
                        cpu_code = gr.Code(label="CPU Code (pandas)", interactive=False)
                        user_code = gr.Code(label="Your GPU Code (cuDF)", language="python")
                        
                        check_btn = gr.Button("✅ Check Solution")
                        benchmark_btn = gr.Button("⚡ Benchmark")
                        
                        puzzle_feedback = gr.Markdown()
                        puzzle_state = gr.State({})
                        
                        def generate_puzzle(diff):
                            puzzle = generate_coding_puzzle(diff)
                            desc = f"**🎯 {diff} Challenge:** {puzzle['description']}"
                            return desc, puzzle['cpu_code'], "", puzzle
                        
                        def check_solution(user_code_input, puzzle):
                            if not puzzle or not user_code_input:
                                return "Generate a puzzle and enter your solution!"
                            
                            similarity = difflib.SequenceMatcher(None, 
                                                               user_code_input.strip(), 
                                                               puzzle['gpu_code'].strip()).ratio()
                            
                            if similarity > 0.8:
                                user_state.points += 10
                                return f"🎉 Excellent! Perfect solution! +10 XP | Total: {user_state.points} XP"
                            elif similarity > 0.6:
                                user_state.points += 5
                                return f"✅ Good effort! Close solution. +5 XP | Total: {user_state.points} XP\n\n💡 **Hint:** Check your imports and variable names"
                            elif similarity > 0.4:
                                user_state.points += 2
                                return f"🟡 Partial credit for trying! +2 XP | Total: {user_state.points} XP\n\n💡 **Hints:**\n• Replace 'pandas' or 'pd' with 'cudf'\n• Check function names and syntax\n• Make sure to import cudf"
                            else:
                                return f"❌ Not quite right. No points awarded.\n\n💡 **Hints:**\n• Replace 'import pandas as pd' with 'import cudf'\n• Replace 'pd.DataFrame' with 'cudf.DataFrame'\n• Replace 'pd.' with 'cudf.' for other operations\n\n**Expected pattern:** {puzzle['gpu_code']}"
                        
                        def run_benchmark(user_code_input, puzzle):
                            if not puzzle:
                                return "Generate a puzzle first!"
                            
                            similarity = difflib.SequenceMatcher(None, 
                                                               user_code_input.strip(), 
                                                               puzzle['gpu_code'].strip()).ratio()
                            
                            if similarity > 0.6:
                                cpu_time = puzzle['cpu_time']
                                gpu_time = puzzle['gpu_time']
                                speedup = round(cpu_time / gpu_time, 1)
                                return f"⚡ **Benchmark Results:**\n📊 CPU: {cpu_time}s\n🚀 GPU: {gpu_time}s\n⚡ **Speedup: {speedup}x**"
                            else:
                                return "❌ Fix your solution first!"
                        
                        gen_puzzle_btn.click(generate_puzzle, inputs=difficulty, 
                                           outputs=[puzzle_desc, cpu_code, user_code, puzzle_state])
                        check_btn.click(check_solution, inputs=[user_code, puzzle_state], outputs=puzzle_feedback)
                        benchmark_btn.click(run_benchmark, inputs=[user_code, puzzle_state], outputs=puzzle_feedback)
                
                # Update progress on point changes
                def refresh_progress():
                    return update_progress()
                
                # Connect progress updates
                gen_quiz_btn.click(refresh_progress, outputs=progress)
                check_btn.click(refresh_progress, outputs=progress)
        
        # Footer
        gr.Markdown("""
        ---
        <div style='text-align:center; color:#666; padding:15px;'>
        🌟 **AI Tutor Complete Learning System** • Structured Learning Paths • Socratic Guidance • Gamified Practice<br>
        📚 <i>From beginner tutorials to advanced GPU computing - your complete data science learning companion!</i><br>
        💡 **Learning Mode:** Topic-based structured courses | **Tutor Mode:** Socratic Q&A | **Game Mode:** XP-based practice
        </div>
        """)
    
    return app

# === LAUNCH ===
if __name__ == "__main__":
    print("\n🚀 Creating AI Tutor interface...")
    app = create_ai_tutor()
    
    print("🎉 AI Tutor ready!")
    print("=" * 60)
    print("✅ Complete Learning System Available:")
    print("  📚 Learning Mode - Structured topic-based learning paths")
    print("  🎓 Tutor Mode - Socratic Q&A with guided discovery")
    print("  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding")
    print("  🌟 Dynamic content generation with contextual quotes")
    print("  📖 Enhanced documentation (12+ sources)")
    print("  🎯 From beginner Python to advanced GPU computing")
    print("=" * 60)
    
    # Launch the app
    app.launch(share=True)


/home/poojapal/.local/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🤖 Loading NousResearch/Hermes-2-Pro-Mistral-7B


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ LLM loaded and ready!
🚀 Initializing RAG system...
🔄 Initializing RAG system...
✅ Loaded cached RAG with 437 chunks

🚀 Creating AI Tutor interface...
🎉 AI Tutor ready!
✅ Complete Learning System Available:
  📚 Learning Mode - Structured topic-based learning paths
  🎓 Tutor Mode - Socratic Q&A with guided discovery
  🎮 Game Mode - XP-based practice with flashcards, quizzes & coding
  🌟 Dynamic content generation with contextual quotes
  📖 Enhanced documentation (12+ sources)
  🎯 From beginner Python to advanced GPU computing
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e6696233e3027eda9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
